In [17]:
import pandas as pd
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.query_api import QueryApi

In [18]:
token = 'MlUmh8wWKDAcBiZbBjvJPguGKxoON1ZRqFjVQ_zh8M3NHLcr53WnxbfXX_ByD0Wm7S8G0kb622gj2EX25MzFCA=='
org = "Portfolio_Opt"
url = "http://localhost:8080"
bucket = "Nifty50"

client = InfluxDBClient(url=url, token=token, org=org)
query_api = client.query_api()

In [19]:
symbols = ['ASIANPAINT.NS', 'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'HCLTECH.NS', 'INFY.NS', 'TCS.NS', 'TECHM.NS']

start_date = '2023-01-01'
end_date = '2024-01-01'

all_data = pd.DataFrame()

In [20]:
for symbol in symbols:
    # Construct the Flux query
    query = f'''
    from(bucket: "{bucket}")
      |> range(start: {start_date}, stop: {end_date})
      |> filter(fn: (r) => r["_measurement"] == "market_data")
      |> filter(fn: (r) => r["_field"] == "close")
      |> filter(fn: (r) => r["symbol"] == "{symbol}")
      |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
    '''
    # Execute the query
    result = query_api.query_data_frame(query=query)
    if not result.empty:
        result.set_index('_time', inplace=True)
        result = result[['close']]  # Ensure only close data is used
        result.columns = [symbol]  # Rename column to symbol name

        if all_data.empty:
            all_data = result
        else:
            all_data = all_data.join(result, how='outer')

print(all_data)

# Save the consolidated data to CSV
all_data.to_csv('backtesting_data.csv')

                           ASIANPAINT.NS  BAJAJ-AUTO.NS  BAJFINANCE.NS  \
_time                                                                    
2023-01-02 00:00:00+00:00    3047.250000    3573.949951    6553.750000   
2023-01-03 00:00:00+00:00    3028.250000    3601.699951    6606.250000   
2023-01-04 00:00:00+00:00    3016.850098    3552.850098    6571.000000   
2023-01-05 00:00:00+00:00    3004.350098    3621.149902    6100.049805   
2023-01-06 00:00:00+00:00    2978.399902    3642.250000    5980.399902   
...                                  ...            ...            ...   
2023-12-22 00:00:00+00:00    3341.300049    6372.100098    7295.299805   
2023-12-26 00:00:00+00:00    3383.350098    6464.549805    7162.299805   
2023-12-27 00:00:00+00:00    3404.449951    6709.649902    7232.799805   
2023-12-28 00:00:00+00:00    3397.250000    6703.299805    7258.649902   
2023-12-29 00:00:00+00:00    3402.399902    6797.250000    7327.750000   

                            HCLTECH.N